# Detect gait
This script use leave-one-subject-out cross-validation (LOSO-CV) to train a classifier on all but one participant (train set) and predict gait on the participant left out (test set). A classifier is subsequently stored by training on all participants.

Execution time: 

### Modules

In [1]:
from pdathome.classification import train_test_gait_detection, store_gait_detection
from pdathome.constants import global_constants as gc

### Constants

In [2]:
l_classifiers = [gc.classifiers.RANDOM_FOREST]

### Process data

In [4]:
from paradigma.gait_analysis_config import GaitFeatureExtractionConfig
from pdathome.load import load_dataframes_directory
from pdathome.classification import cv_train_test_model

subject = 'hbv002' 
l_ids = gc.participant_ids.L_PD_IDS + gc.participant_ids.L_HC_IDS
config_class = GaitFeatureExtractionConfig
l_classifiers = l_classifiers
target_column_name = gc.columns.GAIT_MAJORITY_VOTING
pred_proba_colname = gc.columns.PRED_GAIT_PROBA
pred_colname = gc.columns.PRED_GAIT
step = 'gait'
path_features = gc.paths.PATH_GAIT_FEATURES
path_predictions = gc.paths.PATH_GAIT_PREDICTIONS
n_jobs = -1

# Initialize configuration
config = config_class()

# Define predictors
l_predictors = list(config.d_channels_values.keys())
l_predictors_scale = [x for x in l_predictors if 'dominant' not in x]

df_all_subjects = load_dataframes_directory(
    directory_path=path_features,
    l_ids=l_ids
)

for classifier_name in l_classifiers:

    # Train and test model
    df_test, classification_threshold = cv_train_test_model(
        subject=subject,
        df=df_all_subjects,
        classifier_name=classifier_name,
        l_predictors=l_predictors,
        l_predictors_scale=l_predictors_scale,
        target_column_name=target_column_name,
        pred_proba_colname=pred_proba_colname,
        n_jobs=n_jobs,
        step=step,
    )

In [1]:
for subject in gc.participant_ids.L_PD_IDS + gc.participant_ids.L_PD_IDS:
    train_test_gait_detection(subject, l_classifiers=l_classifiers)

for classifier in l_classifiers:
    store_gait_detection(classifier)